# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [78]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [79]:
# your code here
orders = pd.read_csv("../data/Orders.csv")
orders.drop("Unnamed: 0", axis=1, inplace=True)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [80]:
# SubProblem 1: find the customers expenditure
customers_expenditure = pd.DataFrame(orders.groupby("CustomerID")["amount_spent"].sum())

In [81]:
# SubProblem 2: Setting quantile ranges for VIPS and Preferred
customers_expenditure.amount_spent.quantile([0.75, 0.95, 1])

0.75      1661.640
0.95      5840.182
1.00    280206.020
Name: amount_spent, dtype: float64

In [82]:
# SubProblem 3: Assign the amount_spent values to the VIPS and Preferred
labels1 = ["preferred","vip"]
customers_expenditure = pd.qcut(customers_expenditure["amount_spent"], q=[0.75,0.95,1.], labels=labels1)
customers_expenditure.head(3)

CustomerID
12346          vip
12347    preferred
12348    preferred
Name: amount_spent, dtype: category
Categories (2, object): [preferred < vip]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [83]:
#Filtering the data for the VIP, meaning the ones that spent more than the .95 quantile
customers_VIP = orders[orders.amount_spent >= 5840.182] 
#Counting the number of people by Nationality
customers_VIP['Country'].value_counts()

United Kingdom    7
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [84]:
# #Filtering the data for the preferred, meaning the ones that spent more than the .75 quantile
customers_VIP_preferred = orders[orders.amount_spent >= 1661.640]
customers_VIP_preferred["Country"].value_counts()

United Kingdom    116
EIRE                4
Singapore           4
Japan               3
Netherlands         2
France              2
Australia           1
Name: Country, dtype: int64